# 👥 Customer Segmentation - Advanced Analytics

This notebook focuses on customer segmentation using various clustering techniques and RFM analysis.

## Objectives:
- Implement RFM (Recency, Frequency, Monetary) analysis
- Apply K-Means clustering for customer segmentation
- Explore hierarchical clustering techniques
- Analyze customer segments and create personas
- Develop targeted marketing strategies
- Validate segmentation results


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('../src')
from components.customer_segmentation import CustomerSegmentation

print("📚 Customer segmentation libraries loaded!")

In [ ]:
# Load data
try:
    df = pd.read_csv('../data/processed/cleaned_data.csv')
    print(f"✅ Cleaned dataset loaded: {df.shape}")
except FileNotFoundError:
    print("📊 Loading raw data for processing...")
    try:
        df = pd.read_csv('../data/raw/customer_shopping_data.csv')
    except FileNotFoundError:
        from utils.common import load_sample_data
        df = load_sample_data(n_customers=2000)
    print(f"Dataset shape: {df.shape}")

# Display basic info
print("\n📋 Dataset Overview:")
print(f"Records: {len(df):,}")
if 'Customer ID' in df.columns:
    print(f"Unique Customers: {df['Customer ID'].nunique():,}")
print(f"Features: {df.shape[1]}")

## 1. Data Preparation for Segmentation

In [ ]:
# Prepare features for clustering
def prepare_segmentation_features(df):
    """Prepare features for customer segmentation"""
    features = pd.DataFrame()
    
    # Customer-level aggregation
    if 'Customer ID' in df.columns:
        customer_features = df.groupby('Customer ID').agg({
            'Purchase Amount (USD)': ['sum', 'mean', 'count', 'std'] if 'Purchase Amount (USD)' in df.columns else None,
            'Age': 'first' if 'Age' in df.columns else None,
            'Review Rating': 'mean' if 'Review Rating' in df.columns else None,
            'Category': 'nunique' if 'Category' in df.columns else None
        }).fillna(0)
        
        # Flatten column names
        customer_features.columns = ['_'.join(col).strip() for col in customer_features.columns.values]
        customer_features = customer_features.reset_index()
        
        features = customer_features
    else:
        # Use individual transaction features if no customer ID
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        features = df[numeric_cols].fillna(0)
    
    print(f"📊 Features prepared: {features.shape}")
    print(f"Feature columns: {list(features.columns)}")
    
    return features

features_df = prepare_segmentation_features(df)
display(features_df.head())

In [ ]:
# Create RFM features
def create_rfm_features(df):
    """Create RFM (Recency, Frequency, Monetary) features"""
    
    if 'Customer ID' not in df.columns:
        print("⚠️ Customer ID not found. Creating synthetic RFM data...")
        # Create synthetic RFM data
        n_customers = min(1000, len(df))
        np.random.seed(42)
        
        rfm_df = pd.DataFrame({
            'Customer_ID': [f'CUST_{i:05d}' for i in range(n_customers)],
            'Recency': np.random.exponential(30, n_customers),  # Days since last purchase
            'Frequency': np.random.poisson(5, n_customers),     # Number of purchases
            'Monetary': np.random.exponential(200, n_customers) # Total spend
        })
    else:
        # Calculate actual RFM from data
        reference_date = pd.to_datetime('2024-01-01')  # Use a reference date
        
        if 'Purchase Date' in df.columns:
            df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])
            
            rfm_df = df.groupby('Customer ID').agg({
                'Purchase Date': lambda x: (reference_date - x.max()).days,
                'Customer ID': 'count',
                'Purchase Amount (USD)': 'sum' if 'Purchase Amount (USD)' in df.columns else None
            })
            
            rfm_df.columns = ['Recency', 'Frequency', 'Monetary']
            rfm_df = rfm_df.reset_index()
        else:
            # Use aggregated features as proxy
            if 'Purchase Amount (USD)' in df.columns:
                customer_agg = df.groupby('Customer ID').agg({
                    'Purchase Amount (USD)': ['sum', 'count']
                })
                customer_agg.columns = ['Monetary', 'Frequency']
                customer_agg['Recency'] = np.random.exponential(30, len(customer_agg))
                rfm_df = customer_agg.reset_index()
            else:
                # Fallback to synthetic data
                n_customers = df['Customer ID'].nunique()
                rfm_df = pd.DataFrame({
                    'Customer ID': df['Customer ID'].unique(),
                    'Recency': np.random.exponential(30, n_customers),
                    'Frequency': np.random.poisson(5, n_customers),
                    'Monetary': np.random.exponential(200, n_customers)
                })
    
    print(f"✅ RFM features created for {len(rfm_df)} customers")
    return rfm_df

rfm_data = create_rfm_features(df)
display(rfm_data.head())
display(rfm_data.describe())

## 2. RFM Analysis and Segmentation

In [ ]:
# RFM Scoring and Segmentation
def create_rfm_segments(rfm_df):
    """Create RFM segments based on quintile scoring"""
    
    # Create RFM scores (quintiles)
    rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1])  # Lower recency = higher score
    rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])  # Higher frequency = higher score
    rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])   # Higher monetary = higher score
    
    # Convert to integers
    rfm_df['R_Score'] = rfm_df['R_Score'].astype(int)
    rfm_df['F_Score'] = rfm_df['F_Score'].astype(int)
    rfm_df['M_Score'] = rfm_df['M_Score'].astype(int)
    
    # Create RFM combined score
    rfm_df['RFM_Score'] = rfm_df['R_Score'].astype(str) + rfm_df['F_Score'].astype(str) + rfm_df['M_Score'].astype(str)
    
    # Define segment rules
    def assign_segment(row):
        if row['R_Score'] >= 4 and row['F_Score'] >= 4 and row['M_Score'] >= 4:
            return 'Champions'
        elif row['R_Score'] >= 3 and row['F_Score'] >= 3 and row['M_Score'] >= 3:
            return 'Loyal Customers'
        elif row['R_Score'] >= 4 and row['F_Score'] <= 2:
            return 'New Customers'
        elif row['R_Score'] >= 3 and row['F_Score'] <= 2 and row['M_Score'] >= 3:
            return 'Potential Loyalists'
        elif row['R_Score'] <= 2 and row['F_Score'] >= 3 and row['M_Score'] >= 3:
            return 'At Risk'
        elif row['R_Score'] <= 2 and row['F_Score'] >= 2 and row['M_Score'] <= 2:
            return 'Cannot Lose Them'
        elif row['R_Score'] <= 2 and row['F_Score'] <= 2:
            return 'Lost Customers'
        else:
            return 'Others'
    
    rfm_df['RFM_Segment'] = rfm_df.apply(assign_segment, axis=1)
    
    return rfm_df

# Apply RFM segmentation
rfm_segmented = create_rfm_segments(rfm_data.copy())

# Display segment distribution
segment_dist = rfm_segmented['RFM_Segment'].value_counts()
print("🎯 RFM SEGMENT DISTRIBUTION:")
print("=" * 40)
for segment, count in segment_dist.items():
    percentage = (count / len(rfm_segmented)) * 100
    print(f"{segment:<20}: {count:>4} ({percentage:>5.1f}%)")

# Visualize RFM segments
fig = px.pie(
    values=segment_dist.values,
    names=segment_dist.index,
    title='Customer Segments Distribution (RFM Analysis)',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
# RFM Segment Analysis
def analyze_rfm_segments(rfm_df):
    """Analyze characteristics of each RFM segment"""
    
    segment_analysis = rfm_df.groupby('RFM_Segment').agg({
        'Recency': ['mean', 'median'],
        'Frequency': ['mean', 'median'],
        'Monetary': ['mean', 'median'],
        'RFM_Segment': 'count'
    }).round(2)
    
    segment_analysis.columns = ['Recency_Mean', 'Recency_Median', 'Frequency_Mean', 'Frequency_Median', 
                               'Monetary_Mean', 'Monetary_Median', 'Customer_Count']
    
    return segment_analysis

segment_stats = analyze_rfm_segments(rfm_segmented)
print("📊 SEGMENT CHARACTERISTICS:")
display(segment_stats)

# Create segment comparison visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Average Recency by Segment', 'Average Frequency by Segment', 
                   'Average Monetary by Segment', 'Customer Count by Segment']
)

# Recency
fig.add_trace(
    go.Bar(x=segment_stats.index, y=segment_stats['Recency_Mean'], name='Recency'),
    row=1, col=1
)

# Frequency
fig.add_trace(
    go.Bar(x=segment_stats.index, y=segment_stats['Frequency_Mean'], name='Frequency'),
    row=1, col=2
)

# Monetary
fig.add_trace(
    go.Bar(x=segment_stats.index, y=segment_stats['Monetary_Mean'], name='Monetary'),
    row=2, col=1
)

# Customer Count
fig.add_trace(
    go.Bar(x=segment_stats.index, y=segment_stats['Customer_Count'], name='Count'),
    row=2, col=2
)

fig.update_xaxes(tickangle=45)
fig.update_layout(height=800, title_text="RFM Segment Analysis", showlegend=False)
fig.show()

## 3. K-Means Clustering Analysis

In [ ]:
# Prepare data for K-means clustering
def prepare_clustering_data(rfm_df):
    """Prepare and scale data for clustering"""
    
    # Select features for clustering
    clustering_features = rfm_df[['Recency', 'Frequency', 'Monetary']].copy()
    
    # Handle outliers (cap at 95th percentile)
    for col in clustering_features.columns:
        cap_value = clustering_features[col].quantile(0.95)
        clustering_features[col] = np.where(clustering_features[col] > cap_value, cap_value, clustering_features[col])
    
    # Scale features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(clustering_features)
    scaled_df = pd.DataFrame(scaled_features, columns=clustering_features.columns)
    
    return scaled_df, scaler, clustering_features

scaled_data, scaler, original_features = prepare_clustering_data(rfm_segmented)
print(f"✅ Clustering data prepared: {scaled_data.shape}")
print(f"Features: {list(scaled_data.columns)}")

# Show scaling effect
print("\n📊 SCALING COMPARISON:")
print("Original data:")
display(original_features.describe())
print("\nScaled data:")
display(scaled_data.describe())

In [ ]:
# Find optimal number of clusters
def find_optimal_clusters(data, max_clusters=10):
    """Find optimal number of clusters using elbow method and silhouette score"""
    
    inertias = []
    silhouette_scores = []
    K_range = range(2, max_clusters + 1)
    
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(data)
        
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(data, kmeans.labels_))
    
    return K_range, inertias, silhouette_scores

# Find optimal clusters
k_range, inertias, sil_scores = find_optimal_clusters(scaled_data)

# Plot elbow curve and silhouette scores
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Elbow Method', 'Silhouette Score']
)

# Elbow method
fig.add_trace(
    go.Scatter(x=list(k_range), y=inertias, mode='lines+markers', name='Inertia'),
    row=1, col=1
)

# Silhouette score
fig.add_trace(
    go.Scatter(x=list(k_range), y=sil_scores, mode='lines+markers', name='Silhouette Score'),
    row=1, col=2
)

fig.update_xaxes(title_text="Number of Clusters (k)")
fig.update_yaxes(title_text="Inertia", row=1, col=1)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)
fig.update_layout(height=400, title_text="Optimal Number of Clusters Analysis")
fig.show()

# Find best k based on silhouette score
best_k = k_range[np.argmax(sil_scores)]
best_silhouette = max(sil_scores)

print(f"📊 OPTIMAL CLUSTERING RESULTS:")
print(f"Best number of clusters: {best_k}")
print(f"Best silhouette score: {best_silhouette:.3f}")

# Show silhouette scores for each k
print("\nSilhouette scores by k:")
for k, score in zip(k_range, sil_scores):
    print(f"k={k}: {score:.3f}")

In [ ]:
# Apply K-means clustering with optimal k
optimal_k = best_k
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(scaled_data)

# Add cluster labels to original data
rfm_clustered = rfm_segmented.copy()
rfm_clustered['KMeans_Cluster'] = cluster_labels

# Analyze K-means clusters
kmeans_analysis = rfm_clustered.groupby('KMeans_Cluster').agg({
    'Recency': ['mean', 'std'],
    'Frequency': ['mean', 'std'],
    'Monetary': ['mean', 'std'],
    'KMeans_Cluster': 'count'
}).round(2)

kmeans_analysis.columns = ['Recency_Mean', 'Recency_Std', 'Frequency_Mean', 'Frequency_Std',
                          'Monetary_Mean', 'Monetary_Std', 'Customer_Count']

print(f"🎯 K-MEANS CLUSTERING RESULTS (k={optimal_k}):")
display(kmeans_analysis)

# Visualize clusters in 3D
fig = px.scatter_3d(
    rfm_clustered,
    x='Recency',
    y='Frequency', 
    z='Monetary',
    color='KMeans_Cluster',
    title=f'K-Means Customer Clusters (k={optimal_k})',
    labels={'color': 'Cluster'}
)
fig.show()

# Show cluster distribution
cluster_dist = rfm_clustered['KMeans_Cluster'].value_counts().sort_index()
print(f"\n📊 CLUSTER DISTRIBUTION:")
for cluster, count in cluster_dist.items():
    percentage = (count / len(rfm_clustered)) * 100
    print(f"Cluster {cluster}: {count:>4} customers ({percentage:>5.1f}%)")

## 4. Hierarchical Clustering Analysis

In [ ]:
# Hierarchical clustering
def perform_hierarchical_clustering(data, n_clusters=None, sample_size=500):
    """Perform hierarchical clustering with dendrogram visualization"""
    
    # Sample data for visualization if too large
    if len(data) > sample_size:
        sample_idx = np.random.choice(len(data), sample_size, replace=False)
        data_sample = data.iloc[sample_idx]
        print(f"📊 Using sample of {sample_size} customers for dendrogram visualization")
    else:
        data_sample = data
    
    # Create dendrogram
    linkage_matrix = linkage(data_sample, method='ward')
    
    plt.figure(figsize=(12, 8))
    dendrogram(linkage_matrix, truncate_mode='level', p=5)
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('Sample Index or (Cluster Size)')
    plt.ylabel('Distance')
    plt.show()
    
    # Apply hierarchical clustering to full dataset
    if n_clusters is None:
        n_clusters = optimal_k  # Use same as K-means for comparison
    
    hierarchical = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    hierarchical_labels = hierarchical.fit_predict(data)
    
    return hierarchical_labels

# Perform hierarchical clustering
hierarchical_labels = perform_hierarchical_clustering(scaled_data, optimal_k)

# Add hierarchical cluster labels
rfm_clustered['Hierarchical_Cluster'] = hierarchical_labels

# Analyze hierarchical clusters
hierarchical_analysis = rfm_clustered.groupby('Hierarchical_Cluster').agg({
    'Recency': ['mean', 'std'],
    'Frequency': ['mean', 'std'],
    'Monetary': ['mean', 'std'],
    'Hierarchical_Cluster': 'count'
}).round(2)

hierarchical_analysis.columns = ['Recency_Mean', 'Recency_Std', 'Frequency_Mean', 'Frequency_Std',
                                'Monetary_Mean', 'Monetary_Std', 'Customer_Count']

print(f"🌳 HIERARCHICAL CLUSTERING RESULTS:")
display(hierarchical_analysis)

## 5. Segmentation Comparison and Validation

In [ ]:
# Compare different segmentation methods
def compare_segmentation_methods(df):
    """Compare RFM, K-means, and Hierarchical clustering results"""
    
    comparison_results = {
        'Method': ['RFM Segments', 'K-Means', 'Hierarchical'],
        'Number of Segments': [df['RFM_Segment'].nunique(), 
                              df['KMeans_Cluster'].nunique(), 
                              df['Hierarchical_Cluster'].nunique()],
        'Silhouette Score': []
    }
    
    # Calculate silhouette scores
    methods_data = {
        'RFM': df['RFM_Segment'].astype('category').cat.codes,
        'KMeans': df['KMeans_Cluster'],
        'Hierarchical': df['Hierarchical_Cluster']
    }
    
    for method, labels in methods_data.items():
        try:
            score = silhouette_score(scaled_data, labels)
            comparison_results['Silhouette Score'].append(score)
        except:
            comparison_results['Silhouette Score'].append(np.nan)
    
    comparison_df = pd.DataFrame(comparison_results)
    return comparison_df

# Compare methods
comparison = compare_segmentation_methods(rfm_clustered)
print("🔍 SEGMENTATION METHODS COMPARISON:")
display(comparison)

# Cross-tabulation between methods
print("\n🔄 RFM vs K-Means Segments:")
rfm_kmeans_crosstab = pd.crosstab(rfm_clustered['RFM_Segment'], rfm_clustered['KMeans_Cluster'])
display(rfm_kmeans_crosstab)

print("\n🔄 K-Means vs Hierarchical Clusters:")
kmeans_hierarchical_crosstab = pd.crosstab(rfm_clustered['KMeans_Cluster'], rfm_clustered['Hierarchical_Cluster'])
display(kmeans_hierarchical_crosstab)

In [ ]:
# Visualize segmentation comparison
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['RFM Segments', 'K-Means Clusters', 'Hierarchical Clusters', 'Segment Sizes'],
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'bar'}]]
)

# RFM Segments 3D
for i, segment in enumerate(rfm_clustered['RFM_Segment'].unique()):
    segment_data = rfm_clustered[rfm_clustered['RFM_Segment'] == segment]
    fig.add_trace(
        go.Scatter3d(
            x=segment_data['Recency'],
            y=segment_data['Frequency'],
            z=segment_data['Monetary'],
            mode='markers',
            name=segment,
            showlegend=False
        ),
        row=1, col=1
    )

# K-Means Clusters 3D
for cluster in rfm_clustered['KMeans_Cluster'].unique():
    cluster_data = rfm_clustered[rfm_clustered['KMeans_Cluster'] == cluster]
    fig.add_trace(
        go.Scatter3d(
            x=cluster_data['Recency'],
            y=cluster_data['Frequency'],
            z=cluster_data['Monetary'],
            mode='markers',
            name=f'K-Means {cluster}',
            showlegend=False
        ),
        row=1, col=2
    )

# Hierarchical Clusters 3D
for cluster in rfm_clustered['Hierarchical_Cluster'].unique():
    cluster_data = rfm_clustered[rfm_clustered['Hierarchical_Cluster'] == cluster]
    fig.add_trace(
        go.Scatter3d(
            x=cluster_data['Recency'],
            y=cluster_data['Frequency'],
            z=cluster_data['Monetary'],
            mode='markers',
            name=f'Hierarchical {cluster}',
            showlegend=False
        ),
        row=2, col=1
    )

# Segment sizes comparison
methods = ['RFM', 'K-Means', 'Hierarchical']
segment_counts = [
    len(rfm_clustered['RFM_Segment'].unique()),
    len(rfm_clustered['KMeans_Cluster'].unique()),
    len(rfm_clustered['Hierarchical_Cluster'].unique())
]

fig.add_trace(
    go.Bar(x=methods, y=segment_counts, name='Segment Count'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Segmentation Methods Comparison")
fig.show()

## 6. Customer Personas and Business Insights

In [ ]:
# Create customer personas based on RFM segments
def create_customer_personas(rfm_df):
    """Create detailed customer personas for each segment"""
    
    personas = {
        'Champions': {
            'description': 'Best customers - high value, frequent purchases, recent activity',
            'characteristics': ['High spending', 'Regular purchases', 'Recently active', 'Highly engaged'],
            'strategy': 'Reward and retain',
            'actions': ['VIP treatment', 'Exclusive offers', 'Early access', 'Loyalty rewards']
        },
        'Loyal Customers': {
            'description': 'Consistent customers with good value and regular engagement',
            'characteristics': ['Moderate to high spending', 'Regular engagement', 'Predictable behavior'],
            'strategy': 'Maintain relationship and increase value',
            'actions': ['Cross-sell', 'Upsell', 'Personalization', 'Relationship building']
        },
        'New Customers': {
            'description': 'Recent customers with potential for growth',
            'characteristics': ['Recent first purchase', 'Low frequency', 'Testing the waters'],
            'strategy': 'Nurture and convert to loyal',
            'actions': ['Welcome series', 'Education', 'Support', 'Incentives for repeat purchase']
        },
        'Potential Loyalists': {
            'description': 'Customers showing signs of increased engagement',
            'characteristics': ['Increasing activity', 'Good value potential', 'Responsive to offers'],
            'strategy': 'Develop into loyal customers',
            'actions': ['Targeted offers', 'Engagement campaigns', 'Value demonstration']
        },
        'At Risk': {
            'description': 'Previously good customers showing declining engagement',
            'characteristics': ['Declining frequency', 'Haven\'t purchased recently', 'High historical value'],
            'strategy': 'Win-back campaigns',
            'actions': ['Re-engagement offers', 'Surveys', 'Personal outreach', 'Special incentives']
        },
        'Cannot Lose Them': {
            'description': 'High-value customers at risk of churning',
            'characteristics': ['High historical spend', 'Infrequent recent activity', 'High churn risk'],
            'strategy': 'Immediate intervention required',
            'actions': ['Personal contact', 'Exclusive offers', 'Account management', 'Problem resolution']
        },
        'Lost Customers': {
            'description': 'Customers who have likely churned',
            'characteristics': ['No recent activity', 'Low engagement', 'Minimal historical value'],
            'strategy': 'Reactivation or let go',
            'actions': ['Win-back campaigns', 'Surveys', 'Minimal investment', 'Learning from churn']
        }
    }
    
    return personas

# Generate personas
personas = create_customer_personas(rfm_segmented)

# Display persona details for each segment found in data
active_segments = rfm_segmented['RFM_Segment'].unique()

print("👥 CUSTOMER PERSONAS ANALYSIS")
print("=" * 60)

for segment in active_segments:
    if segment in personas:
        persona = personas[segment]
        segment_size = (rfm_segmented['RFM_Segment'] == segment).sum()
        segment_pct = (segment_size / len(rfm_segmented)) * 100
        
        print(f"\n🎯 {segment.upper()} ({segment_size} customers, {segment_pct:.1f}%)")
        print(f"Description: {persona['description']}")
        print(f"Strategy: {persona['strategy']}")
        print("Key Actions:")
        for action in persona['actions']:
            print(f"  • {action}")
        print("-" * 50)

In [ ]:
# Calculate business impact metrics
def calculate_business_impact(rfm_df):
    """Calculate business impact metrics for each segment"""
    
    segment_impact = rfm_df.groupby('RFM_Segment').agg({
        'Monetary': ['sum', 'mean'],
        'Frequency': 'mean',
        'Recency': 'mean',
        'RFM_Segment': 'count'
    }).round(2)
    
    segment_impact.columns = ['Total_Revenue', 'Avg_Revenue_Per_Customer', 'Avg_Frequency', 'Avg_Recency', 'Customer_Count']
    
    # Calculate revenue share
    total_revenue = segment_impact['Total_Revenue'].sum()
    segment_impact['Revenue_Share_Pct'] = (segment_impact['Total_Revenue'] / total_revenue * 100).round(1)
    
    # Calculate customer share
    total_customers = segment_impact['Customer_Count'].sum()
    segment_impact['Customer_Share_Pct'] = (segment_impact['Customer_Count'] / total_customers * 100).round(1)
    
    # Sort by total revenue
    segment_impact = segment_impact.sort_values('Total_Revenue', ascending=False)
    
    return segment_impact

# Calculate business impact
business_impact = calculate_business_impact(rfm_segmented)

print("💼 BUSINESS IMPACT ANALYSIS")
print("=" * 50)
display(business_impact)

# Visualize business impact
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Revenue Share by Segment', 'Customer Share by Segment', 
                   'Revenue vs Customer Share', 'Average Revenue per Customer']
)

# Revenue share
fig.add_trace(
    go.Bar(x=business_impact.index, y=business_impact['Revenue_Share_Pct'], name='Revenue Share'),
    row=1, col=1
)

# Customer share
fig.add_trace(
    go.Bar(x=business_impact.index, y=business_impact['Customer_Share_Pct'], name='Customer Share'),
    row=1, col=2
)

# Revenue vs Customer Share scatter
fig.add_trace(
    go.Scatter(
        x=business_impact['Customer_Share_Pct'], 
        y=business_impact['Revenue_Share_Pct'],
        mode='markers+text',
        text=business_impact.index,
        textposition='top center',
        name='Segments'
    ),
    row=2, col=1
)

# Average revenue per customer
fig.add_trace(
    go.Bar(x=business_impact.index, y=business_impact['Avg_Revenue_Per_Customer'], name='Avg Revenue'),
    row=2, col=2
)

fig.update_xaxes(tickangle=45)
fig.update_layout(height=800, title_text="Business Impact by Customer Segment", showlegend=False)
fig.show()

In [ ]:
# Save segmentation results
try:
    # Save detailed segmentation results
    rfm_clustered.to_csv('../reports/analysis/03_customer_segmentation_results.csv', index=False)
    print("💾 Segmentation results saved to ../reports/analysis/03_customer_segmentation_results.csv")
    
    # Save business impact analysis
    business_impact.to_csv('../reports/analysis/03_business_impact_analysis.csv')
    print("💾 Business impact analysis saved to ../reports/analysis/03_business_impact_analysis.csv")
    
    # Save segmentation comparison
    comparison.to_csv('../reports/analysis/03_segmentation_comparison.csv', index=False)
    print("💾 Segmentation comparison saved to ../reports/analysis/03_segmentation_comparison.csv")
    
    print("\n✅ Customer segmentation analysis completed successfully!")
    print("🚀 Ready for next notebook: 04_predictive_modeling.ipynb")
    
except Exception as e:
    print(f"⚠️ Could not save reports: {e}")
    print("📊 Analysis completed - results available in notebook")

# Final summary
print("\n" + "=" * 60)
print("📋 SEGMENTATION ANALYSIS SUMMARY")
print("=" * 60)
print(f"✅ RFM Analysis: {rfm_segmented['RFM_Segment'].nunique()} segments identified")
print(f"✅ K-Means Clustering: {optimal_k} clusters (silhouette score: {best_silhouette:.3f})")
print(f"✅ Hierarchical Clustering: {rfm_clustered['Hierarchical_Cluster'].nunique()} clusters")
print(f"✅ Business Impact: Revenue concentration analysis completed")
print(f"✅ Customer Personas: Actionable strategies developed")
print("\n🎯 Key Recommendation: Focus on Champions and At-Risk segments for maximum impact")